In [1]:
import geemap
import ee

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
Map.add_basemap("CartoDB.DarkMatter")

In [4]:
area_shp= "2025_Exam_RS_study_area\study_area.shp"
area = geemap.shp_to_ee(area_shp)
Map.addLayer(area)

The input shapefile could not be found.


In [5]:
geometry = area
import ee
import geemap

# Initializing Earth Engine
ee.Initialize()

# Reading the uploaded shapefile for area of interest
geometry = ee.FeatureCollection('projects/rmtumon/assets/study_area').geometry()

# Using Sentinel-2 1C collection instead of 2A
sen = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
    .filterBounds(geometry) \
    .filterDate('2015', '2025') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .map(lambda img: img.select('B.*').multiply(0.0001) \
        .normalizedDifference(['B8', 'B4']) \
        .rename('ndvi') \
        .copyProperties(img, ['system:time_start', 'system:time_end']))

# Comparing before and after periods
before = sen.filterDate('2016','2017').median().rename('before').clip(geometry)
after = sen.filterDate('2018','2019').median().rename('after').clip(geometry)

# Creating tile layers for the split map
left_layer = geemap.ee_tile_layer(before, {'min': 0, 'max': 1, 'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']}, "NDVI 2015-2022")
right_layer = geemap.ee_tile_layer(after, {'min': 0, 'max': 1, 'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']}, "NDVI 2023-2024")

# Creating a split map
Map = geemap.Map()
Map.split_map(left_layer, right_layer)
Map.addLayer(geometry, {}, 'Study Area')  # Optional: Add study area boundary
Map.centerObject(geometry, zoom=10)
Map

Map(center=[1.888871750732988, -73.73118463829267], controls=(ZoomControl(options=['position', 'zoom_in_text',…